In [3]:
from general_session_creator import create_session

session = create_session()

In [6]:
from DP_transacciones_total import get_transacciones_dp_total
from snowflake.snowpark import DataFrame
import snowflake.snowpark.functions as fn
from datetime import date, timedelta

transacciones_dp_total = get_transacciones_dp_total(session)

def usuarios_por_numero_de_transacciones(transacciones: DataFrame) -> DataFrame:
    return (
        transacciones
        .group_by('EMAIL')
        .agg(fn.count_distinct('ORDER_ID').alias('TRANSACCIONES'))
        .with_column(
            'NUMERO_DE_TRANSACCIONES',
            fn.when(fn.col('TRANSACCIONES') > 10, 11)
            .otherwise(fn.col('TRANSACCIONES'))
        )
        .group_by('NUMERO_DE_TRANSACCIONES')
        .agg(fn.count_distinct('EMAIL').alias('USUARIOS'))
        .order_by('NUMERO_DE_TRANSACCIONES')
    )

transacciones_180_dias_corte_octubre = (
    transacciones_dp_total
    .filter(fn.col('FECHA') <= date.fromisoformat('2023-10-29'))
    .filter(date.fromisoformat('2023-10-29') - timedelta(180) <= fn.col('FECHA'))
)

transacciones_180_dias_corte_abril = (
    transacciones_dp_total
    .filter(fn.col('FECHA') <= date.fromisoformat('2023-04-30'))
    .filter(date.fromisoformat('2023-04-30') - timedelta(180) <= fn.col('FECHA'))
)

usuarios_por_numero_de_transacciones(transacciones_180_dias_corte_abril).to_pandas().to_csv('txn_180_corte_abril.csv', index = False)
usuarios_por_numero_de_transacciones(transacciones_180_dias_corte_octubre).to_pandas().to_csv('txn_180_corte_octubre.csv', index = False)